# Test FileMonitor Cache Operations

This notebook tests the file monitoring functionality of AutoCoderRAGDocListener, including:
- File addition
- File modification
- File deletion

In [1]:
import os
import tempfile
import shutil
import time
from autocoder.rag.variable_holder import VariableHolder
from tokenizers import Tokenizer

# Initialize tokenizer
VariableHolder.TOKENIZER_PATH = "/Users/allwefantasy/Downloads/tokenizer.json"
VariableHolder.TOKENIZER_MODEL = Tokenizer.from_file(VariableHolder.TOKENIZER_PATH)

from autocoder.rag.cache.file_monitor_cache import AutoCoderRAGDocListener

# Create a temporary directory for testing
test_dir = tempfile.mkdtemp()
print(f"Created test directory: {test_dir}")

Created test directory: /var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq


In [2]:
# Initialize the cache manager
cache_manager = AutoCoderRAGDocListener(
    path=test_dir,
    ignore_spec=None,
    required_exts=[".txt", ".py"]
)

# Give it some time to initialize
time.sleep(2)

2024-12-17 14:54:54.246 | INFO     | autocoder.rag.cache.file_monitor_cache:open_watch:90 - start monitor: /var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq...


In [3]:
# Test file addition
def test_file_addition():
    # Create a test file
    test_file = os.path.join(test_dir, "w1.txt")
    with open(test_file, "w") as f:
        f.write("This is a test file")
    
    # Wait for the file monitor to detect the change
    time.sleep(2)
    
    # Get cache and check if file is added
    cache = cache_manager.get_cache()
    print("\nCache after adding file:")
    for file_path, data in cache.items():
        print(f"File: {file_path}")
        print(f"Content: {data['content']}")

test_file_addition()

2024-12-17 14:55:06.092 | INFO     | autocoder.rag.utils:process_file_local:124 - Load file /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w1.txt in 0.0007789134979248047
2024-12-17 14:55:06.093 | INFO     | autocoder.rag.cache.file_monitor_cache:update_cache:81 - update cache: /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w1.txt
2024-12-17 14:55:06.093 | INFO     | autocoder.rag.cache.file_monitor_cache:update_cache:82 - current cache: dict_keys(['/private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w1.txt'])
2024-12-17 14:55:15.598 | INFO     | autocoder.rag.utils:process_file_local:124 - Load file /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w1.txt in 0.0003781318664550781
2024-12-17 14:55:15.599 | INFO     | autocoder.rag.cache.file_monitor_cache:update_cache:81 - update cache: /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w1.txt
2024-12-17 14:55:15.599 | INFO     | autocoder


Cache after adding file:
File: /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w1.txt
Content: [{'module_name': '##File: /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w1.txt', 'source_code': 'This is a test file', 'tag': '', 'tokens': 13, 'metadata': {}}]


In [4]:
# Test file modification
def test_file_modification():
    # Modify the test file
    test_file = os.path.join(test_dir, "w1.txt")
    with open(test_file, "w") as f:
        f.write("This is a modified test file")
    
    # Wait for the file monitor to detect the change
    time.sleep(2)
    
    # Get cache and check if file content is updated
    cache = cache_manager.get_cache()
    print("\nCache after modifying file:")
    for file_path, data in cache.items():
        print(f"File: {file_path}")
        print(f"Content: {data['content']}")

test_file_modification()


Cache after modifying file:
File: /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w1.txt
Content: [{'module_name': '##File: /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w1.txt', 'source_code': 'This is a modified test file', 'tag': '', 'tokens': 14, 'metadata': {}}]


In [6]:
# Test file deletion
def test_file_deletion():
    # Delete the test file
    test_file = os.path.join(test_dir, "w1.txt")
    os.remove(test_file)
    
    # Wait for the file monitor to detect the change
    time.sleep(2)
    
    # Get cache and verify file is removed
    cache = cache_manager.get_cache()
    print("\nCache after deleting file:")
    print(f"Files in cache: {list(cache.keys())}")

test_file_deletion()


Cache after deleting file:
Files in cache: []


In [8]:
# Test with Python files
def test_python_file():
    # Create a Python test file
    test_file = os.path.join(test_dir, "w.py")
    with open(test_file, "w") as f:
        f.write("def hello():\n    print('Hello World!')")
    
    # Wait for the file monitor to detect the change
    time.sleep(2)
    
    # Get cache and check if file is added
    cache = cache_manager.get_cache()
    print("\nCache after adding Python file:")
    for file_path, data in cache.items():
        print(f"File: {file_path}")
        print(f"Content: {data['content']}")

test_python_file()


Cache after adding Python file:
File: /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w.py
Content: [{'module_name': '##File: /private/var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq/w.py', 'source_code': "def hello():\n    print('Hello World!')", 'tag': '', 'tokens': 19, 'metadata': {}}]


In [9]:
# Cleanup
def cleanup():
    cache_manager.stop()
    shutil.rmtree(test_dir)
    print(f"\nCleaned up test directory: {test_dir}")

cleanup()


Cleaned up test directory: /var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/tmphovefknq
